In [1]:
import cv2
import os
from datetime import date
from datetime import datetime
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import joblib

In [20]:
face_detector = cv2.CascadeClassifier('model/haarcascade_frontalface_default.xml')
smile_detector = cv2.CascadeClassifier('model/haarcascade_smile.xml')
eye_detector = cv2.CascadeClassifier('model/haarcascade_eye.xml')

In [3]:
cap = cv2.VideoCapture(0)

In [4]:
datetoday = date.today().strftime("%m_%d_%y")
datetoday2 = date.today().strftime("%d-%B-%Y")

In [5]:
if not os.path.isdir('Attendance'):
    os.makedirs('Attendance')
if not os.path.isdir('static/faces'):
    os.makedirs('static/faces')
if f'Attendance-{datetoday}.csv' not in os.listdir('Attendance'):
    with open(f'Attendance/Attendance-{datetoday}.csv','w') as f:
        f.write('Name,Roll,Time')


In [6]:
def totalreg():
    return len(os.listdir('static/faces'))

In [7]:
def extract_faces(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_points = face_detector.detectMultiScale(gray, 1.3, 5)
    return face_points

In [8]:
def smile_detect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    smile_points = smile_detector.detectMultiScale(gray, 1.7, 35)
    return smile_points

In [21]:
def eye_detect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eye_points = eye_detector.detectMultiScale(gray, 1.3, 5)
    return eye_points

In [9]:
def identify_face(facearray):
    model = joblib.load('model/face_recognition_model.pkl')
    return model.predict(facearray)

In [10]:
def train_model():
    faces = []
    labels = []
    userlist = os.listdir('static/faces')
    for user in userlist:
        for imgname in os.listdir(f'static/faces/{user}'):
            img = cv2.imread(f'static/faces/{user}/{imgname}')
            resized_face = cv2.resize(img, (50, 50))
            faces.append(resized_face.ravel())
            labels.append(user)
    faces = np.array(faces)
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(faces,labels)
    joblib.dump(knn,'model/face_recognition_model.pkl')

In [11]:
def extract_attendance():
    df = pd.read_csv(f'Attendance/Attendance-{datetoday}.csv')
    names = df['Name']
    rolls = df['Roll']
    times = df['Time']
    l = len(df)
    return names,rolls,times,l

In [12]:
def add_attendance(name):
    username = name.split('_')[0]
    userid = name.split('_')[1]
    current_time = datetime.now().strftime("%H:%M:%S")
    
    df = pd.read_csv(f'Attendance/Attendance-{datetoday}.csv')
    if int(userid) not in list(df['Roll']):
        with open(f'Attendance/Attendance-{datetoday}.csv','a') as f:
            f.write(f'\n{username},{userid},{current_time}')

In [13]:
names,rolls,times,l = extract_attendance()

In [14]:
names,rolls,times,l

(Series([], Name: Name, dtype: object),
 Series([], Name: Roll, dtype: object),
 Series([], Name: Time, dtype: object),
 0)

In [15]:
def draw_smile(img, center, radius):
    axes = (radius, radius//2)
    color = (255,0,0)
    cv2.ellipse(img, center, axes, 0, 0, 180, color,thickness=2,lineType=cv2.LINE_AA)
    return

In [16]:
# def detection(grayscale, face, face_detect, smile_detect):
#     #NGUYEN BAN
    
#     face = face_detect.detectMultiScale(grayscale, 1.3, 5)
#     for (x,y,w,h) in face:

#         ri_grayscale = grayscale[y:y+h, x:x+w]
#         ri_color = face[y:y+h, x:x+w]
        
        
#         cv2.rectangle(face,(x, y), (x+w, y+h), (255, 0, 20), 2)
        
#         smile_lst = smile_detect.detectMultiScale(ri_grayscale, 1.7, 35)
#         smiled = False
        
#         for (x_smile, y_smile, w_smile, h_smile) in smile_lst:
#             draw_smile(ri_color, (x_smile + w_smile // 2, y_smile + h_smile // 4), radius=w_smile // 3)
#             smiled =  True
            
#         if smiled:
#             # cv2.putText(face,"Smile",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,20),2)
#             cv2.putText(face,"Smile",(100,100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),3)
            
#     return face

In [17]:
# def detection(grayscale, img, face_detect, smile_detect):
    
#     count = 0
    
#     face = face_detect.detectMultiScale(grayscale, 1.3, 5)
    
#     while ret:
        
#         if extract_faces(img)!=():
#             (x,y,w,h) = extract_faces(img)[0]

#         # ri_color = img[y:y+h, x:x+w]
        
#         cv2.rectangle(img,(x, y), (x+w, y+h), (255, 0, 20), 2)
    
#         face = cv2.resize(img[y:y+h, x:x+w], (50, 50))
#         identified_person = identify_face(face.reshape(1,-1))[0]
#         add_attendance(identified_person)
#         cv2.putText(img,f'{identified_person}',(30,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 20, 20),2,cv2.LINE_AA)
        
#         smile_lst = smile_detect.detectMultiScale(img, 1.7, 35)
#         smiled = False
        
#         for (x_smile, y_smile, w_smile, h_smile) in smile_lst:
#             # draw_smile(ri_color, (x_smile + w_smile // 2, y_smile + h_smile // 4), radius=w_smile // 3)
#             smiled =  True
            
#         if smiled:
#             # cv2.putText(face,"Smile",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,20),2)
#             cv2.putText(img,"Smile",(100,100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),3)
#             count += 1
#             print(count)
#     return img

In [22]:
import dlib  # for face and landmark detection
import imutils

ModuleNotFoundError: No module named 'dlib'

In [18]:
# cap = cv2.VideoCapture(0)
# ret = True
# count = 0
# while ret:
#     ret,frame = cap.read()
    
#     face = extract_faces(frame)
#     smile = smile_detect(frame)
    
#     for (x_face, y_face, w_face, h_face) in face:
        

        
#         cv2.rectangle(frame,(x_face, y_face), (x_face+w_face, y_face+h_face), (255, 0, 20), 2)
#         face = cv2.resize(frame[y_face:y_face+h_face,x_face:x_face+w_face], (50, 50))
#         identified_person = identify_face(face.reshape(1,-1))[0]
        
        
#         ri_color = frame[y_face:y_face+h_face, x_face:x_face+w_face]
#         smiled = False
        
#         for (x_smile, y_smile, w_smile, h_smile) in smile:
#             draw_smile(ri_color, (x_smile + w_smile // 2, y_smile + h_smile // 4), radius=w_smile // 3)
#             smiled =  True

#         if smiled:
#             cv2.putText(frame,"Smile!",(30,90),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)
#             count += 1
            
            
#         cv2.putText(frame,f'{identified_person}',(30,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 20, 20),2,cv2.LINE_AA)
#         cv2.putText(frame,f'Frame Smiling: {count}/200',(30,60),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 100, 100),2,cv2.LINE_AA)
        
        

#     cv2.imshow('Attendance',frame)
    
#     # smile_lst = smile_detector.detectMultiScale(frame, 1.7, 35)
    
#     if cv2.waitKey(1)==27:
#         break
    
#     if count == 200:
#         add_attendance(identified_person)
#         break
# print(count)

# cap.release()
# cv2.destroyAllWindows()
# names,rolls,times,l = extract_attendance()  

200


In [19]:
# cap = cv2.VideoCapture(0)
# ret = True

# while ret:
#     ret,frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     frame = detection(gray, frame,face_detector, smile_detector)

#     cv2.imshow('Attendance',frame)
#     if cv2.waitKey(1)==27:
#         break
    
# cap.release()
# cv2.destroyAllWindows()

In [26]:
cap = cv2.VideoCapture(0)
ret = True

while ret:
    ret,frame = cap.read()
    
    face = extract_faces(frame)
    smile = smile_detect(frame)
    
    for (x_face, y_face, w_face, h_face) in face:

        cv2.rectangle(frame,(x_face, y_face), (x_face+w_face, y_face+h_face), (255, 0, 20), 2)
        cv2.putText(frame,f'{x_face, y_face, w_face, h_face}',(x_face,y_face-20),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 20, 20),2,cv2.LINE_AA)
        
        face = cv2.resize(frame[y_face:y_face+h_face,x_face:x_face+w_face], (50, 50))
        identified_person = identify_face(face.reshape(1,-1))[0]
        
    
    cv2.imshow('Attendance',frame)
    
    # smile_lst = smile_detector.detectMultiScale(frame, 1.7, 35)
    
    if cv2.waitKey(1)==27:
        break

cap.release()
cv2.destroyAllWindows()
names,rolls,times,l = extract_attendance()  